In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence = 0.8, min_tracking_confidence = 0.5)

In [3]:
IMAGESET_FOLDER = os.path.join('Imageset')
actions = np.array(['A', 'B', 'J', 'N', 'W', "teacher", "live", "Kol", "sign", "language"])
no_sequences = 30
no_frames = 30

In [4]:
def mediapipe_detection(image, holistic):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = holistic.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, result

In [5]:
def render_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
cap = cv2.VideoCapture(0)

for action in range(6, 7):
    for seqNo in range(no_sequences):
        for frameNo in range(no_frames):
            ret, frame = cap.read()
            
            h, w, c = frame.shape

            backBlackImage = np.zeros([h, w, c])

            backBlackImage.fill(0)
            
            image, results = mediapipe_detection(frame, holistic)
        
            render_landmarks(backBlackImage, results)
            
            image = cv2.flip(image, 1)
            backBlackImage = cv2.flip(backBlackImage, 1)

            if frameNo == 0:
                cv2.putText(backBlackImage, "STARTING COLLECTION FOR {}".format(actions[action]), (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4, cv2.LINE_AA)
                cv2.putText(backBlackImage, "\"{}\" - ({}/15)".format(actions[action], seqNo + 1), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, "\"{}\" - ({}/15)".format(actions[action], seqNo + 1), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.imshow("Live Feed", image)
                cv2.imshow("Extracted Hand", backBlackImage)
                #cv2.imshow("Shifted Hands", shiftedOriginImage)
                cv2.waitKey(1500)
            else:
                cv2.putText(backBlackImage, "\"{}\" - ({}/15)".format(actions[action], seqNo + 1), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, "\"{}\" - ({}/15)".format(actions[action], seqNo + 1), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
                cv2.imshow("Live Feed", image)
                cv2.imshow("Extracted Hand", backBlackImage)
                #cv2.imshow("Shifted Hands", shiftedOriginImage)
                
            image_save_path = os.path.join(IMAGESET_FOLDER, actions[action], str(seqNo))
            
            try:
                os.makedirs(image_save_path)
            except:
                pass

            cv2.imwrite(os.path.join(image_save_path, str(frameNo) + '.jpg'), image)
            
            cv2.waitKey(10)
            
cap.release()
cv2.destroyAllWindows()